In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 3, Finished, Available)

In [3]:
dframe_sales = pd.read_excel("abfss://Tejas_MDW_FinalProj@onelake.dfs.fabric.microsoft.com/TMDW_FinalProj.Lakehouse/Files/C/Sales*.xlsx")

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 5, Finished, Available)

In [4]:
#creating Spark data frame for sales
dframe1 = spark.createDataFrame(dframe_sales)

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 6, Finished, Available)

In [5]:
# Displaying the Data Frame
display(dframe1.head(16))

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 7, Finished, Available)

SynapseWidget(Synapse.DataFrame, bd520652-0ca2-4709-9d3b-5bd5d3afb22d)

In [7]:
dframe_RefundLog = pd.read_excel("abfss://Tejas_MDW_FinalProj@onelake.dfs.fabric.microsoft.com/TMDW_FinalProj.Lakehouse/Files/C/Sales*.xlsx",sheet_name="Refund Log")

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 9, Finished, Available)

In [8]:
dframe2 = spark.createDataFrame(dframe_RefundLog)

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 10, Finished, Available)

In [9]:
# displaying the Refund Log data
display(dframe2.head(16))

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 11, Finished, Available)

SynapseWidget(Synapse.DataFrame, 6e04cb01-4b81-424b-9f6c-26c44ad8c6c2)

In [24]:
dframe = dframe1.join(dframe2,dframe1.OID==dframe2.OID,how="left").drop(dframe2.OID,dframe2.Cust_Name,dframe2.Amount)

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 26, Finished, Available)

In [28]:
dframe_modified = dframe.withColumn("Order_Year", year("Order_Date")) \
                        .withColumn("Order_Month", month("Order_Date")) \
                        .withColumn("Created_TimeStamp", current_timestamp()) \
                        .withColumn("Modified_TimeStamp", current_timestamp())


StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 30, Finished, Available)

In [45]:
#creating a view to load the data
dframe_modified.createOrReplaceTempView("View_Sales")

StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 47, Finished, Available)

In [46]:
%%sql
create table if not exists TMDW_FinalProj.BronzeSales
(
    OID	string,
    Order_Date	Date,
    Ship_Date	date,
    Aging	int,
    Ship_Mode	string,
    Product_Category	string,
    Product	 string,
    Sales 	float,
    Quantity	float,
    Discount	 float,
    Profit 	 float,
    Ship_Cost 	float,
    Order_Priority	string,
    Cust_ID	string,
    Cust_Name	string,
    Segment	string,
    City	string,
    State	string,
    Country	string,
    Region string,
	Return string,
    Order_Year int,
    Order_Month int,
    Created_TS  TIMESTAMP,
    Modified_TS  TIMESTAMP
)
using DELTA
PARTITIONED by(Order_Year,Order_Month)


StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 48, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [50]:
%%sql
Merge into  TMDW_FinalProj.BronzeSales as MBS
using View_Sales as VS
on  MBS.Order_Year=VS.Order_Year and MBS.Order_Month=VS.Order_Month and MBS.OID=VS.OID
when matched then 
update SET
MBS.Order_Date	=	VS.Order_Date	,
MBS.Ship_Date	=	VS.Ship_Date	,
MBS.Aging	=	VS.Aging	,
MBS.Ship_Mode	=	VS.Ship_Mode	,
MBS.Product_Category	=	VS.Product_Category	,
MBS.Product	=	VS.Product	,
MBS.Sales 	=	VS.Sales 	,
MBS.Quantity	=	VS.Quantity	,
MBS.Discount	=	VS.Discount	,
MBS.Profit 	=	VS.Profit 	,
MBS.Ship_Cost 	=	VS.Ship_Cost 	,
MBS.Order_Priority	=	VS.Order_Priority	,
MBS.Cust_ID	=	VS.Cust_ID	,
MBS.Cust_Name	=	VS.Cust_Name	,
MBS.Segment	=	VS.Segment	,
MBS.City	=	VS.City	,
MBS.State	=	VS.State	,
MBS.Country	=	VS.Country	,
MBS.Region  	=	VS.Region  	,
MBS.Return  	=	VS.Returned  	,
MBS.Modified_TS	=	VS.Modified_TimeStamp	

when not matched then 
INSERT
(
MBS.OID,    
MBS.Order_Date	,
MBS.Ship_Date	,
MBS.Aging	,
MBS.Ship_Mode	,
MBS.Product_Category	,
MBS.Product	,
MBS.Sales 	,
MBS.Quantity	,
MBS.Discount	,
MBS.Profit 	,
MBS.Ship_Cost 	,
MBS.Order_Priority	,
MBS.Cust_ID	,
MBS.Cust_Name	,
MBS.Segment	,
MBS.City	,
MBS.State	,
MBS.Country	,
MBS.Region  	,
MBS.Return,
MBS.Order_Year  	,
MBS.Order_Month  	,
MBS.Created_TS   	,
MBS.Modified_TS	
)
values
(
VS.OID,
VS.Order_Date	,
VS.Ship_Date	,
VS.Aging	,
VS.Ship_Mode	,
VS.Product_Category	,
VS.Product	,
VS.Sales 	,
VS.Quantity	,
VS.Discount	,
VS.Profit 	,
VS.Ship_Cost 	,
VS.Order_Priority	,
VS.Cust_ID	,
VS.Cust_Name	,
VS.Segment	,
VS.City	,
VS.State	,
VS.Country	,
VS.Region  	,
VS.Returned,
VS.Order_Year  	,
VS.Order_Month  	,
VS.Created_TimeStamp   	,
VS.Modified_TimeStamp	

)


StatementMeta(, e2ad1e5d-92b7-414c-a9d5-9568d5a1afaa, 52, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>